## 1. Setup & Installation

In [ ]:
# Install required packages (run once)
# Uncomment the lines below if you haven't installed these packages

# Core packages
# !pip install transformers datasets accelerate bitsandbytes

# For LoRA fine-tuning
# !pip install peft trl

# For fast fine-tuning (recommended!)
# !pip install unsloth

# Database connection
# !pip install psycopg2-binary python-dotenv

In [ ]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv(Path('..') / '.env')

print("Environment loaded!")
print(f"HF Token available: {'HF_TOKEN' in os.environ or 'HUGGINGFACE_TOKEN' in os.environ}")
print(f"Neon DB available: {'NEON_DATABASE_URL' in os.environ}")

## 2. Export Training Data from Neon

In [ ]:
import psycopg2
from psycopg2.extras import RealDictCursor

def get_training_data():
    """Fetch training examples from Neon database."""
    conn = psycopg2.connect(os.environ['NEON_DATABASE_URL'])
    cur = conn.cursor(cursor_factory=RealDictCursor)
    
    # Get summary first
    cur.execute("SELECT * FROM training_data_summary")
    summary = cur.fetchone()
    print("=" * 50)
    print("Training Data Summary")
    print("=" * 50)
    if summary:
        print(f"Total interactions: {summary.get('total_interactions', 0)}")
        print(f"Approved: {summary.get('approved_count', 0)}")
        print(f"Rejected: {summary.get('rejected_count', 0)}")
        print(f"Corrected: {summary.get('corrected_count', 0)} ← GOLD DATA!")
        print(f"Training examples: {summary.get('training_examples', 0)}")
    else:
        print("No training data yet. Run the curator agent with HITL!")
    
    # Get training examples
    cur.execute("""
        SELECT instruction, input, output, quality_score
        FROM training_examples
        WHERE quality_score >= 0.7
        ORDER BY created_at DESC
    """)
    examples = cur.fetchall()
    
    cur.close()
    conn.close()
    
    return examples

training_examples = get_training_data()
print(f"\nHigh-quality training examples: {len(training_examples)}")

In [ ]:
# Preview a few examples
if training_examples:
    print("Sample Training Example:")
    print("-" * 50)
    example = training_examples[0]
    print(f"Instruction: {example['instruction'][:200]}...")
    print(f"Input: {example['input'][:200] if example['input'] else 'N/A'}...")
    print(f"Output: {example['output'][:200]}...")
    print(f"Quality: {example['quality_score']}")
else:
    print("No training examples yet!")
    print("\nTo generate training data:")
    print("1. Run: python curator-agent/run_with_approval.py")
    print("2. When prompted, choose [e]dit to provide corrections")
    print("3. Your corrections become high-quality training examples")

## 3. Prepare Dataset for Fine-Tuning

In [ ]:
from datasets import Dataset

def format_for_training(examples):
    """Convert to Alpaca-style format for fine-tuning."""
    formatted = []
    for ex in examples:
        # Alpaca format with input field
        if ex['input']:
            text = f"""### Instruction:
{ex['instruction']}

### Input:
{ex['input']}

### Response:
{ex['output']}"""
        else:
            text = f"""### Instruction:
{ex['instruction']}

### Response:
{ex['output']}"""
        
        formatted.append({"text": text})
    
    return formatted

# Create dataset
if training_examples:
    formatted_data = format_for_training(training_examples)
    dataset = Dataset.from_list(formatted_data)
    print(f"Dataset created with {len(dataset)} examples")
    print("\nSample formatted text:")
    print(dataset[0]['text'][:500] + "...")
else:
    print("Need training examples first!")
    dataset = None

In [ ]:
# Save dataset locally for backup
output_dir = Path('../data/training')
output_dir.mkdir(parents=True, exist_ok=True)

if dataset:
    # Save as JSONL
    jsonl_path = output_dir / 'proves_training_data.jsonl'
    with open(jsonl_path, 'w') as f:
        for example in formatted_data:
            f.write(json.dumps(example) + '\n')
    print(f"Saved to {jsonl_path}")
    
    # Also save as HF dataset
    dataset.save_to_disk(str(output_dir / 'proves_dataset'))
    print(f"Saved HF dataset to {output_dir / 'proves_dataset'}")

## 4. Fine-Tune with Unsloth (Fast + Memory Efficient)

Unsloth provides 2x faster fine-tuning with 50% less memory.

**Hardware Requirements:**
- 1B model: 4GB VRAM (RTX 3060 or better)
- 7B model: 16GB VRAM (RTX 4090 or A100)
- Or use Google Colab with T4 GPU (free tier works for small models!)

In [ ]:
# Check if we have GPU
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Recommend model based on VRAM
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    if vram_gb >= 16:
        print("\n✅ Recommended: unsloth/mistral-7b-instruct-v0.3")
    elif vram_gb >= 8:
        print("\n✅ Recommended: HuggingFaceTB/SmolLM2-1.7B-Instruct")
    else:
        print("\n✅ Recommended: unsloth/Llama-3.2-1B-Instruct")
else:
    print("No GPU detected!")
    print("Options:")
    print("1. Use Google Colab (free T4 GPU)")
    print("2. Use CPU (very slow, not recommended)")
    print("3. Rent GPU from vast.ai, runpod.io, or lambda labs")

In [ ]:
# Fine-tuning code (requires GPU)
# Uncomment when ready to train

'''
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments

# Load base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct",  # Change based on your VRAM
    max_seq_length=2048,
    dtype=None,  # Auto-detect
    load_in_4bit=True,  # QLoRA - uses less VRAM
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

# Training config
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=100,  # Increase for more training
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    output_dir="outputs",
    optim="adamw_8bit",
    seed=42,
)

# Trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    args=training_args,
)

# Train!
trainer.train()
'''

print("Fine-tuning code ready - uncomment when you have training data and GPU!")

## 5. Push to Hugging Face Hub

In [ ]:
# Login to Hugging Face
from huggingface_hub import login, HfApi

# Get token from environment or prompt
hf_token = os.environ.get('HF_TOKEN') or os.environ.get('HUGGINGFACE_TOKEN')

if hf_token:
    login(token=hf_token)
    api = HfApi()
    user_info = api.whoami()
    print(f"Logged in as: {user_info['name']}")
else:
    print("No HF token found. Set HF_TOKEN in .env or run:")
    print("  from huggingface_hub import login")
    print("  login()")

In [ ]:
# Push dataset to Hub (optional)
'''
if dataset:
    dataset.push_to_hub(
        "Elizo/proves-cubesat-dependencies",
        private=True  # Set to False to share publicly
    )
    print("Dataset pushed to Hugging Face Hub!")
'''

# Push fine-tuned model to Hub (after training)
'''
model.push_to_hub(
    "Elizo/proves-dependency-extractor-lora",
    token=hf_token,
)
tokenizer.push_to_hub(
    "Elizo/proves-dependency-extractor-lora",
    token=hf_token,
)
print("Model pushed to Hugging Face Hub!")
'''

print("Push code ready - uncomment after training!")

## 6. Test the Fine-Tuned Model

In [ ]:
# Test inference (after training)
'''
# Switch to inference mode
FastLanguageModel.for_inference(model)

# Test prompt
test_prompt = """### Instruction:
Extract dependencies from the following documentation and assess criticality.

### Input:
The ImuManager component initializes during boot and requires the LinuxI2cDriver
to communicate with the BMX160 IMU sensor. If I2C fails, the spacecraft cannot
determine its orientation for pointing solar panels at the sun.

### Response:
"""

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=256)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)
'''

print("Inference code ready - uncomment after training!")

## 📊 Training Data Pipeline

```
┌─────────────────────────────────────────────────────────────┐
│  CURATOR AGENT (Claude)                                     │
│     │                                                       │
│     ▼                                                       │
│  Extracts dependencies from docs                            │
│     │                                                       │
│     ▼                                                       │
│  HITL: Human reviews HIGH criticality                       │
│     │                                                       │
│     ├─[Approve]──► training_interactions (confirmed)        │
│     ├─[Reject]───► training_interactions (negative)         │
│     └─[CORRECT]──► training_interactions (GOLD!) ◄── BEST   │
│                           │                                 │
│                           ▼                                 │
│                    training_examples                        │
│                    (Alpaca format)                          │
│                           │                                 │
│                           ▼                                 │
│                    This notebook exports                    │
│                           │                                 │
│                           ▼                                 │
│                    Fine-tune local LLM                      │
│                    (Llama/Mistral/Qwen)                     │
│                           │                                 │
│                           ▼                                 │
│                    Push to Hugging Face                     │
│                    (your own model!)                        │
└─────────────────────────────────────────────────────────────┘
```

## Next Steps

1. **Collect more training data** - Run the curator agent and provide corrections
2. **Get GPU access** - Use Colab, Lambda Labs, or your own GPU
3. **Fine-tune** - Start with a small model like Llama-3.2-1B
4. **Evaluate** - Test on held-out examples
5. **Iterate** - Collect more data, retrain, improve